In [1]:
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(path)
if path not in sys.path:
    sys.path.append(path)
import pandas as pd
import numpy as np
from custom_transformers.CustomTransformers import Cleaner , ReplyFinder , QuestionClassifier ,InformationClassifier
from custom_transformers.CustomTransformers import CombineMessageReply  
from custom_transformers.CustomTransformers import GetLocationList,GetStatusList,FilterIsGivingInformation
from custom_transformers.CustomTransformers import FilterHasLoc,FilterHasStat,GenerateSummary,NerClassifier,FilterLen
from constants import COLUMNS_WE_NEED
from utils.transform_data import transform_reply_to
from utils.clean_data import clean_text
from sklearn.pipeline import Pipeline



c:\Users\nasser\Desktop\RoadProject\BackEnd\Road


c:\Users\nasser\.conda\envs\finall\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nasser\.conda\envs\finall\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('../data/raw.csv')

In [3]:
df.head(1)

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
0,Message,428911,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:04:46+00:00,الدي سي اوه عليه مشاكل ؟؟,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0


In [4]:
# get only the first 2 rows as df 
df = df.head(1)

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import sys
import os
import joblib

from utils.helpers import combine_sub_words, word2features
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
from keras.models import load_model
   
from utils.clean_data import clean_text
from constants import COLUMNS_WE_NEED,HAWAJEZ_NAMES,STATUS_NAMES,STATUS_MAPPER
from utils.transform_data import transform_reply_to
import tensorflow as tf
import pyarabic.araby as araby
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from fuzzywuzzy import fuzz
import jellyfish


In [9]:
pep = Pipeline([('cleaner', Cleaner()), ('reply_finder', ReplyFinder()),
                     ("question_classifier",QuestionClassifier()),("information_classifier",InformationClassifier())
                     ,("combine_message_reply",CombineMessageReply()),("filter",FilterIsGivingInformation()),
                     ("ner_crf_classifier",NerClassifier()),("Locations",GetLocationList()),("Status",GetStatusList()),
                     ("filter_has_loc",FilterHasLoc()),("filter_has_stat",FilterHasStat()),("filter_len",FilterLen()),("summary",GenerateSummary())])
result = pep.fit_transform(df)
# result

Current step: Cleaner
Current step: ReplyFinder
Current step: QuestionClassifier
1/1 [==============================] - 0s 26ms/step
Current step: InformationClassifier
1/1 [==============================] - 0s 463ms/step
Current step: CombineMessageReply
Current step: FilterIsGivingInformation
Current step: NerClassifier
1/1 [==============================] - 1s 1s/step
Current step: GetLocationList
Current step: GetStatusList
Current step: FilterHasLoc
Current step: FilterHasStat
Current step: FilterLen
Current step: GenerateSummary


In [8]:
# save the pipeline 
joblib.dump(pep, '../models/pipeline/pipeline.pkl')


['../models/pipeline/pipeline.pkl']

In [9]:
# load the pipeline
loaded_pipeline = joblib.load('../models/pipeline/pipeline.pkl')

In [10]:
df

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
0,Message,428911,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:04:46+00:00,الدي سي اوه عليه مشاكل ؟؟,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
1,Message,428913,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:12:41+00:00,شباب طريق جنين قلقيلة شو وضعها,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
2,Message,428918,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:20:54+00:00,حومش و شافي شمرون شو وضعهم,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
3,Message,428920,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:24:52+00:00,صرة للداخل نابلس؟,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
4,Message,428922,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:28:31+00:00,من رام الله لنابلس,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Message,429014,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 13:25:23+00:00,شافي شمرون كيف وضعها,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
96,Message,429015,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 13:28:34+00:00,شباب شو وضع الكونتينر,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
97,Message,429016,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 13:29:00+00:00,طريق دير ابزيع كفر نعمة مغلق قي مواجهات,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0
98,Message,429017,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 13:29:13+00:00,NaN,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0


In [11]:
loaded_pipeline.fit_transform(df)

Current step: Cleaner
Current step: ReplyFinder
Current step: QuestionClassifier
3/3 [==============================] - 0s 3ms/step
Current step: InformationClassifier
3/3 [==============================] - 0s 6ms/step
Current step: CombineMessageReply
Current step: FilterIsGivingInformation
Current step: NerClassifier
2/2 [==============================] - 1s 22ms/step
Current step: GetLocationList
Current step: GetStatusList
Current step: FilterHasLoc
Current step: FilterHasStat
Current step: FilterLen
Current step: GenerateSummary


c:\Users\nasser\Desktop\RoadProject\BackEnd\Road\custom_transformers\CustomTransformers.py:427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Stat'] = X['Stat'].apply(self.map_stat)
c:\Users\nasser\Desktop\RoadProject\BackEnd\Road\custom_transformers\CustomTransformers.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Stat'] = X['Stat'].apply(self.solve_conflit)


[{'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'قلنديا', 'status': 'مفتوح'},
 {'location': 'قلنديا', 'status': 'ازمة'},
 {'location': 'شافي شمرون', 'status': 'مفتوح'},
 {'location': 'الحمراء', 'status': 'مفتوح'},
 {'location': 'الحمراء', 'status': 'ازمة'},
 {'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'صرة', 'status': 'مفتوح'},
 {'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'بيت فوريك', 'status': 'مفتوح'},
 {'location': 'جيت', 'status': 'مفتوح'},
 {'location': 'صرة', 'status': 'مفتوح'},
 {'location': 'جيت', 'status': 'ازمة'},
 {'location': 'بيت فوريك', 'status': 'مفتوح'},
 {'location': 'قلنديا', 'status': 'مفتوح'},
 {'location': 'قلنديا', 'status': 'ازمة'},
 {'location': 'المربعة', 'status': 'مفتوح'},
 {'location': 'زعترة', 'status': 'مفتوح'},
 {'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'زعترة', 'status': 'مفتوح'},
 {'location': 'حوارة', 'status': 'مفتوح'},
 {'location': 'دير شرف', 'status': 'مسكر'},